In [1]:
import tensorflow 
from tensorflow.keras.datasets import mnist

2022-06-11 00:17:14.796126: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-11 00:17:14.796157: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [3]:
import matplotlib.pyplot as plt 
import random

In [4]:
from tensorflow.keras.utils import to_categorical

In [5]:
def preprocess_data(X_train, y_train, X_test, y_test):
	# reshape images to the the required size by Keras
  	X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
  	X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)
  	# convert from integers to floats
  	X_train = X_train.astype('float32')
  	X_test = X_test.astype('float32')
  	# normalize to range 0-1
  	X_train = X_train/255.0
  	X_test_norm = X_test/255.0
  	# One-hot encoding label 
  	y_train = to_categorical(y_train)
  	y_test = to_categorical(y_test)
  	return X_train, y_train, X_test, y_test 

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
#from tensorflow.keras.optimizers import SGD
from tensorflow.keras.metrics import categorical_crossentropy

In [7]:
def LeNet():
  model = Sequential()
  model.add(Conv2D(filters = 6, kernel_size = (5,5), padding = 'same', activation = 'relu', input_shape = (28,28,1)))
  model.add(MaxPooling2D(pool_size = (2,2)))
  model.add(Conv2D(filters = 16, kernel_size = (5,5), activation = 'relu'))
  model.add(MaxPooling2D(pool_size = (2,2)))
  model.add(Flatten())
  model.add(Dense(120, activation = 'relu'))
  model.add(Dense(10, activation = 'softmax'))
  # compile the model with a loss function, a metric and an optimizer function
  #opt = SGD(lr = 0.001, momentum = 0.99, nesterov = True)
  #opt = Apollo(learning_rate=0.01, epsilon=1e-6, rebound='belief')
  #opt = SGD(learning_rate=0.002, momentum=0.99, nesterov = True)
  model.compile(loss = categorical_crossentropy, 
                optimizer = 'adam', 
                metrics = ['accuracy']) 
  return model

In [8]:
LeNet_model = LeNet()
LeNet_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 6)         156       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 6)        0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 10, 10, 16)        2416      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 16)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 400)               0         
                                                                 
 dense (Dense)               (None, 120)               4

2022-06-11 00:17:16.899489: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-11 00:17:16.899528: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-11 00:17:16.899554: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ruslan-Inspiron-5770): /proc/driver/nvidia/version does not exist
2022-06-11 00:17:16.899750: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
def train_model(model, X_train, y_train, X_test, y_test, epochs = 10, batch_size = 128):
  	# Rescaling all training and testing data
  	X_train, y_train, X_test, y_test = preprocess_data(X_train, y_train, X_test, y_test)
  	# Fitting the model
  	history = model.fit(X_train, y_train, epochs = epochs, batch_size = batch_size, steps_per_epoch = X_train.shape[0]//batch_size, validation_data = (X_test, y_test), validation_steps = X_test.shape[0]//batch_size, verbose = 1)
  	# evaluate the model
  	_, acc = model.evaluate(X_test, y_test, verbose = 1)
  	print('%.3f' % (acc * 100.0))
  	summary_history(history)

In [10]:
train_model(LeNet_model, X_train, y_train, X_test, y_test)

Epoch 1/10


2022-06-11 00:17:17.301441: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 188160000 exceeds 10% of free system memory.


468/468 [==============================] - 11s 23ms/step - loss: 0.3166 - accuracy: 0.9095 - val_loss: 11.7743 - val_accuracy: 0.9714
Epoch 2/10
468/468 [==============================] - 12s 25ms/step - loss: 0.0798 - accuracy: 0.9752 - val_loss: 6.8669 - val_accuracy: 0.9824
Epoch 3/10
468/468 [==============================] - 12s 27ms/step - loss: 0.0569 - accuracy: 0.9829 - val_loss: 5.8115 - val_accuracy: 0.9857
Epoch 4/10
468/468 [==============================] - 12s 25ms/step - loss: 0.0456 - accuracy: 0.9860 - val_loss: 5.1525 - val_accuracy: 0.9879
Epoch 5/10
468/468 [==============================] - 12s 25ms/step - loss: 0.0376 - accuracy: 0.9882 - val_loss: 5.9091 - val_accuracy: 0.9873
Epoch 6/10
468/468 [==============================] - 12s 25ms/step - loss: 0.0309 - accuracy: 0.9901 - val_loss: 5.9989 - val_accuracy: 0.9877
Epoch 7/10
468/468 [==============================] - 11s 25ms/step - loss: 0.0272 - accuracy: 0.9914 - val_loss: 4.4705 - val_accuracy: 0.9898
Ep

NameError: name 'summary_history' is not defined